In [1]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [2]:
text = open('../content/KakaoTalk_20200311_1733_24_621_장윤호.txt','rb').read().decode(encoding='utf-8')
print('Text Length : {}'.format(len(text)))

Text Length : 840185


In [3]:
print(text[:250])

장윤호 님과 카카오톡 대화
저장한 날짜 : 2020-03-11 17:33:27

--------------- 2019년 9월 15일 일요일 ---------------
[장윤호] [오후 4:54] 0퍼대 ㅊㅊ
[장윤호] [오후 4:54] 내몫까지
[장윤호] [오후 4:54] 즐겨줘
[정영준] [오후 5:35] 2연패..
[정영준] [오후 5:35] 너도빨리 플래 ㄱ
[장윤호] [오후 5:39] 난 다른것들을하려고
[장윤호] [


In [0]:
vocab=sorted(set(text))

In [6]:
print('Unique letters : {}'.format(len(vocab)))

Unique letters : 1624


In [8]:
vocab[:20]

['\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2']

In [0]:
import numpy as np
char2idx = {u:i for i,u in enumerate(vocab)} #{'아':3}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) # text to int

In [12]:
for char,_ in zip(char2idx,range(20)):
    print('{}:{}'.format(repr(char),char2idx[char]))

'\n':0
'\r':1
' ':2
'!':3
'"':4
'#':5
'%':6
'&':7
"'":8
'(':9
')':10
'*':11
'+':12
',':13
'-':14
'.':15
'/':16
'0':17
'1':18
'2':19


In [13]:
print('{} ---- mapping to int --> {}'.format(repr(text[:15]),text_as_int[:15]))

'장윤호 님과 카카오톡 대화\r' ---- mapping to int --> [1183 1143 1574    2  419  212    2 1362 1362 1089 1445    2  440 1582
    1]


In [15]:
seq_length = 100
examples_per_epoch = len(text)/seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

장
윤
호
 
님


In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)# tf.data.Dataset.from_tensor_slices.batch 개수 남으면 버리기  
# seq_length+1  batch to  trainset 1~100   testset 2~101

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'장윤호 님과 카카오톡 대화\r\n저장한 날짜 : 2020-03-11 17:33:27\r\n\r\n--------------- 2019년 9월 15일 일요일 ---------------\r\n[장윤'
'호] [오후 4:54] 0퍼대 ㅊㅊ\r\n[장윤호] [오후 4:54] 내몫까지\r\n[장윤호] [오후 4:54] 즐겨줘\r\n[정영준] [오후 5:35] 2연패..\r\n[정영준] [오후 5:35'
'] 너도빨리 플래 ㄱ\r\n[장윤호] [오후 5:39] 난 다른것들을하려고\r\n[장윤호] [오후 5:39] 축구도하고\r\n[장윤호] [오후 5:39] 유튜브도보고\r\n[장윤호] [오후 5:3'
'9] 드라마도보고\r\n[정영준] [오후 8:18] 좋지\r\n[정영준] [오후 8:18] 나도\r\n[정영준] [오후 8:18] 담주부턴\r\n[정영준] [오후 8:18] 롤 막\r\n[정영준] ['
'오후 8:18] 하루6시개\r\n[정영준] [오후 8:18] 간\r\n[정영준] [오후 8:18] 평일에못함ㅋㅋ\r\n[정영준] [오후 8:18] 진짜학교생활시작\r\n[정영준] [오후 8:18]'


In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text,target_text

dataset = sequences.map(split_input_target) 
print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [20]:
for input_example,target_example in dataset.take(1):
    print('input data:',repr(''.join(idx2char[input_example.numpy()])))
    print('output data:',repr(''.join(idx2char[target_example.numpy()])))

input data: '장윤호 님과 카카오톡 대화\r\n저장한 날짜 : 2020-03-11 17:33:27\r\n\r\n--------------- 2019년 9월 15일 일요일 ---------------\r\n[장'
output data: '윤호 님과 카카오톡 대화\r\n저장한 날짜 : 2020-03-11 17:33:27\r\n\r\n--------------- 2019년 9월 15일 일요일 ---------------\r\n[장윤'


In [25]:
for i,(input_idx,target_idx) in enumerate(zip(input_example[:5],target_example[:5])):
    print("{:4d} iter".format(i))
    print(" input: {} ({:s})".format(input_idx,idx2char[input_idx]))
    print(" prediction: {} ({:s})".format(target_idx,idx2char[target_idx]))

   0 iter
 input: 1183 (장)
 prediction: 1143 (윤)
   1 iter
 input: 1143 (윤)
 prediction: 1574 (호)
   2 iter
 input: 1574 (호)
 prediction: 2 ( )
   3 iter
 input: 2 ( )
 prediction: 419 (님)
   4 iter
 input: 419 (님)
 prediction: 212 (과)


In [27]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)

embedding_dim = 512

rnn_units = 1024


In [0]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
        tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(vocab_size=vocab_size,embedding_dim=embedding_dim,rnn_units=rnn_units,batch_size=BATCH_SIZE)

In [37]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 1624)


In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           831488    
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (64, None, 1624)          1664600   
Total params: 8,791,640
Trainable params: 8,791,640
Non-trainable params: 0
_________________________________________________________________


In [39]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 794, 1436,  828,  453, 1115,  852, 1595,   90,  303,  470,  848,
       1078, 1131, 1525,   24, 1230, 1344, 1280,  954,  484,   88, 1372,
       1198,  775,  294,  701, 1277,  755,  711, 1454, 1541,  109,  865,
       1380,  833,  585,  512,  679, 1141,  762,  820, 1158,  808,  743,
        262, 1329, 1044, 1429,  400,  821,  735,  359,  433,  643, 1383,
        961,  297,  274,  840,  512,  483,  514,  701,  546, 1020, 1041,
        191,  786,  739, 1023, 1594,  634,  342,  625, 1373,   39,  393,
       1272,   79, 1144,  730, 1565, 1522, 1242, 1158,  399,  167, 1179,
       1146,  386,  937,  767, 1197,  737, 1012,  621, 1365,   37,  175,
       1273])

In [40]:
print('input: \n',repr(''.join(idx2char[input_example_batch[0]])))
print('\nprediction for next letter: \n',repr(''.join(idx2char[sampled_indices])))

input: 
 '정영준] [오후 7:05] ㅁㄹ\r\n[정영준] [오후 7:05] 5판3선이니\r\n[장윤호] [오후 7:05] 오 5판3선임?\r\n[정영준] [오후 7:05] Ig  vs  그리핀\r\n[정'

prediction for next letter: 
 '봬털빴덤욕뿌훅z꾹돕뽄열웨핀7죠출쩝슴됬x컥절벼꼽몬쩍밴묹퉁핵ㄸ사케뺄랴딜먼유벌빚의브밝까쳐얀탤뉴빛및넓닭른켰시꽃깸뻑딜됩딥몬뙤악앵겧볼밖않후룻낸롸컨F누째o율밀혁픔쥐의뉘갴잠융농쉘벙젇바씅롬캄D건쨌'


In [43]:
def loss(labels,logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True) #softmax

example_batch_loss = loss(target_example_batch,example_batch_predictions)
print(example_batch_predictions.shape)
print(example_batch_loss.numpy().mean())

(64, 100, 1624)
7.3933654


In [0]:
import os
model.compile(loss=loss,optimizer='Adam')
checkpoint_dir='./training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [56]:
EPOCHS=50
history = model.fit(dataset,epochs=EPOCHS,callbacks=[checkpoint_callback])

Train for 129 steps
Epoch 1/50
129/129 [==============================] - 18s 143ms/step - loss: 2.7058
Epoch 2/50
129/129 [==============================] - 18s 137ms/step - loss: 1.5713
Epoch 3/50
129/129 [==============================] - 18s 139ms/step - loss: 1.4628
Epoch 4/50
129/129 [==============================] - 18s 138ms/step - loss: 1.4079
Epoch 5/50
129/129 [==============================] - 18s 140ms/step - loss: 1.3699
Epoch 6/50
129/129 [==============================] - 18s 140ms/step - loss: 1.3413
Epoch 7/50
129/129 [==============================] - 18s 143ms/step - loss: 1.3180
Epoch 8/50
129/129 [==============================] - 18s 141ms/step - loss: 1.2976
Epoch 9/50
129/129 [==============================] - 18s 141ms/step - loss: 1.2789
Epoch 10/50
129/129 [==============================] - 18s 143ms/step - loss: 1.2618
Epoch 11/50
129/129 [==============================] - 18s 143ms/step - loss: 1.2464
Epoch 12/50
129/129 [==============================] -

In [57]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [58]:
model = build_model(vocab_size,embedding_dim,rnn_units,batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 512)            831488    
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           6295552   
_________________________________________________________________
dense_5 (Dense)              (1, None, 1624)           1664600   
Total params: 8,791,640
Trainable params: 8,791,640
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model,start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval,0)

    text_generated=[]
    temperature = 0.80

    model.reset_states() #stateful = True 로 생성된 초기 state 초기화
    for i in range(num_generate):
        predictions = model(input_eval)

        predictions = tf.squeeze(predictions,0)

        predictions = predictions/temperature  # reduce greedy
        predicted_id = tf.random.categorical(predictions,1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id],0)

        text_generated.append(idx2char[predicted_id])

    return (start_string+''.join(text_generated))



In [86]:
print(generate_text(model,start_string=u"[장윤호] [오전"))

[장윤호] [오전 2:18] 그렇게얘기하냐
[정영준] [오전 1:27] 전화이나
[장윤호] [오전 1:00] 아침부터
[정영준] [오전 2:51] ㅋㅋㅋ
[정영준] [오전 1:36] 해도안됨
[장윤호] [오전 3:02] ㅋㅋㅋㅋㅋㅋ
[장윤호] [오전 2:37] 학교도다니느라
[장윤호] [오전 5:24] 자주이뭐냐
[정영준] [오전 2:35] ㅋㅋㅋㅋㅋㅋㅋㅋㅋ
[장윤호] [오전 2:26] 먹으면되네
[정영준] [오전 12:51] 개덥지도
[장윤호] [오전 1:03] 진욱이 형 안녕 
[장윤호] [오전 1:08] 공부를하기로
[장윤호] [오전 2:30] 일단그냥
[정영준] [오전 2:02] 막 8시부터 2시까지 11시경때
[장윤호] [오전 2:33] 문과면
[장윤호] [오전 12:46] 나도
[장윤호] [오전 1:38] 하니까
[장윤호] [오후 7:57] 그냥 교수님들으로
[정영준] [오후 9:23] 아니 세계지
[정영준] [오후 11:05] 고전파
[장윤호] [오전 1:30] 오오전 2:48] 탄사람이
[정영준] [오전 2:30] 진호가
[정영준] [오전 1:57] 시스템종료창
[정영준] [오전 12:32] 너무싫네
[정영준] [오전 5:25] ..
[장윤호] [오전 12:30] 가져가서
[장윤호] [오전 3:43] ㄹㅇ
[정영준] [오전 2:50] ?????
[정영준] [오전 12:12] 사랑을했다~
[정영준] [오전 12:48] 안하나 [오후 4:30] 초콜릿보다
[장윤호] [오후 3:46] 시원하지
[정영준] [오후 6:06] 그렇구나
[정영준] [오후 7:38] 이건
[정영준] [오후 7:56] 판수가야돼서
[장윤호] [오후 2:51] 너의소확히도
[장윤호] [오후 5:23] ㅇㅇ
[정영준] [오후 8:11] 그렇게
[정영준] [오후 7:05] 근데 너무
[정영준] [오후 6:17] 이지랄하는겆
[정영준] [오후 10:54] 나가잖앜
[장윤호] [오전 2:49] 어디서
[정영준] [오전 1:05]
